In [2]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
        


In [3]:
import simpy

env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [4]:
class Car(object):
    
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            yield self.env.process(self.charge(charge_duration))
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
            
    def charge(self, duration):
         yield self.env.timeout(duration)


In [5]:
import simpy
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [6]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [7]:
class Car(object):
    
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('Was interrupted. Hope, the battery is full enough ...')
                
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
            
    def charge(self, duration):
         yield self.env.timeout(duration)

In [8]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


In [9]:
def car(env, name, bcs, driving_time, charge_duration):
    yield env.timeout(driving_time)
    
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [10]:
import simpy

env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [11]:
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [12]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12
